<a href="https://colab.research.google.com/github/jonclindaniel/LargeScaleComputing_S20/blob/master/Labs/Lab%206%20PySpark%20EDA%20and%20ML%20in%20an%20EMR%20Notebook/Local_Colab_Spark_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark in a Colab Notebook

You can run Spark both locally and on a cluster. Here, I'll demonstrate how you can set up Spark to run in a Colab notebook for debugging purposes. You can also set up Spark locally in this same way if you want to take advantage of multiple CPU cores on your laptop (the setup will vary slightly, though, depending on your operating system and you'll need to figure out these specifics on your own; however, this setup does also work on WSL for me).

This being said, this local option should be for testing purposes on sample datasets only. If you want to run big PySpark jobs, you will want to run these in an EMR notebook (with an EMR cluster as your backend).

First, we need to install Spark and PySpark, by running the following commands:

In [0]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
tar -xzf spark-2.4.5-bin-hadoop2.7.tgz
pip install pyspark findspark

OK, now that we have Spark, we need to set a path to it, so PySpark knows where to find it. We do this using the `os` Python library below.

On my machine (WSL, Ubuntu 20.04), where I unpacked Spark in my home directory, this can be achieved with:
```
os.environ["SPARK_HOME"] = "~/spark-2.4.5-bin-hadoop2.7"
```

In Colab, it is automatically downloaded to the `/content` directory, so we indicate that as its location here. Then, we run `findspark` to find Spark for us on the machine, and finally start up a SparkSession running on all available cores (`local[4]` means your code will run on 4 threads locally, `local[*]` means that your code will run as many threads as there are logical cores on your machine.


In [0]:
# Set path to Spark
import os
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Find Spark so that we can access session within our notebook
import findspark
findspark.init()

# Start SparkSession on all available cores
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Now that we've installed everything and set up our paths correctly, we can run (small) Spark jobs both in Colab notebooks and locally (for bigger jobs, you will want to run these jobs on an EMR cluster, though. Remember, for instance, that Google only allocates us one CPU core for free)!

Let's make sure our setup is working by doing couple of simple things with the pyspark.sql package on the Amazon Customer Review Sample Dataset.

In [0]:
! pip install wget
import wget

wget.download('https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv', 'sample_data/sample_us.tsv')

'sample_data/sample_us.tsv'

In [0]:
# Read TSV file from default data download directory in Colab
data = spark.read.csv('sample_data/sample_us.tsv',
                      sep="\t",
                      header=True,
                      inferSchema=True)

In [0]:
data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [0]:
(data.groupBy('star_rating')
     .sum('total_votes')
     .sort('star_rating', ascending=False)
     .show()
)

+-----------+----------------+
|star_rating|sum(total_votes)|
+-----------+----------------+
|          5|              13|
|          4|               3|
|          3|               8|
|          2|               2|
|          1|               8|
+-----------+----------------+

